In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, AutoModel, T5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq, AutoModelForCausalLM
import torch
import json
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
import itertools
import re
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\alexr\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\alexr\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\alexr\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\alexr\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.core.multiarray failed to import

In [5]:
# Название модели
MODEL_NAME = "cointegrated/rubert-tiny2"

# Загружаем токенизатор и модель (сначала без указания меток)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Загружаем JSON-датасет
dataset = load_dataset("json", data_files="computer_faq_extended_dataset.json")

# Создаём словарь меток: каждое уникальное значение ответа получает свой идентификатор
unique_answers = list(set([item["ответ"] for item in dataset["train"]]))
label2id = {answer: idx for idx, answer in enumerate(unique_answers)}
id2label = {idx: answer for answer, idx in label2id.items()}

# Загружаем модель с указанием количества классов и сопоставлением меток
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(unique_answers),
    id2label=id2label,
    label2id=label2id
)

# Отправляем модель на GPU, если он доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Токенизация
def tokenize_function(examples):
    return tokenizer(examples["вопрос"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Добавляем числовые метки
def add_labels(examples):
    examples["label"] = [label2id[ans] for ans in examples["ответ"]]
    return examples

tokenized_datasets = tokenized_datasets.map(add_labels, batched=True)

# Разделяем датасет на обучающую и тестовую выборки
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

# Загружаем метрики
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

# Функция вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"],
    }

# Настройки обучения
training_args = TrainingArguments(
    output_dir="./models/rubert_tiny_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
    logging_dir="./logs",
    load_best_model_at_end=True,  # Загружает лучшую модель после обучения
    metric_for_best_model="f1",  # Ориентируемся на f1-score
    greater_is_better=True,
)

# Создаём Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Запускаем обучение
trainer.train()

# Оцениваем модель на тестовой выборке
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/39 [00:00<?, ? examples/s]

C:\Users\alexr\anaconda3\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


C:\Users\alexr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\alexr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\alexr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\alexr\anaconda3\Lib\site-packages\sklea

Evaluation Results: {'eval_loss': 3.7146029472351074, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 0.5939, 'eval_samples_per_second': 6.735, 'eval_steps_per_second': 1.684, 'epoch': 3.0}


C:\Users\alexr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\alexr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
def answer_question(question):
    # Токенизируем вопрос
    inputs = tokenizer(question, padding="max_length", truncation=True, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Получаем логиты
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Получаем предсказанную метку
    predicted_class_id = logits.argmax().item()
    # Преобразуем числовую метку обратно в ответ
    predicted_answer = id2label[predicted_class_id]
    return predicted_answer

# Пример вызова функции:
question = "Почему компьютер не включается?"
print("Вопрос:", question)
print("Ответ:", answer_question(question))


Вопрос: Почему компьютер не включается?
Ответ: Отключите лишние программы в автозагрузке, проверьте диск на ошибки.


In [15]:
# Задаём название модели RuBERT-tiny
MODEL_NAME = "cointegrated/rubert-tiny2"

# Загружаем токенайзер и модель RuBERT-tiny для получения эмбеддингов
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Функция получения эмбеддингов для текста
def get_embedding(text):
    # Токенизируем текст
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Берём среднее значение по токенам (mean pooling)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()[0]

# Загружаем датасет FAQ
dataset = load_dataset("json", data_files="computer_faq_extended_dataset.json")["train"]

# Строим базу: для каждого вопроса сохраняем эмбеддинг и ответ
faq_database = []
for item in dataset:
    question_text = item["вопрос"]
    answer_text = item["ответ"]
    emb = get_embedding(question_text)
    faq_database.append({
        "question": question_text,
        "answer": answer_text,
        "embedding": emb
    })

# Функция для поиска ближайшего вопроса в базе с помощью косинусного сходства
def find_best_answer(user_question):
    user_emb = get_embedding(user_question)
    # Сравниваем эмбеддинг пользователя с эмбеддингами FAQ
    embeddings = np.array([entry["embedding"] for entry in faq_database])
    scores = cosine_similarity([user_emb], embeddings)[0]
    best_idx = int(np.argmax(scores))
    best_match = faq_database[best_idx]
    return best_match["answer"], scores[best_idx]

# Функция для обработки запроса
def answer_question(user_question):
    answer, score = find_best_answer(user_question)
    return answer, score

# Пример использования
if __name__ == "__main__":
    user_question = "Почему компьютер не включается?"
    answer, score = answer_question(user_question)
    print("Вопрос:", user_question)
    print("Найденный ответ:", answer)
    print("Схожесть:", score)

Вопрос: Почему компьютер не включается?
Найденный ответ: Проверьте кабели, блок питания и исправность розетки.
Схожесть: 0.9999999


In [19]:
# Пример использования
if __name__ == "__main__":
    user_question = "монитор не показывает изображение, что мне делать?"
    answer, score = answer_question(user_question)
    print("Вопрос:", user_question)
    print("Найденный ответ:", answer)
    print("Схожесть:", score)

Вопрос: монитор не показывает изображение, что мне делать?
Найденный ответ: Проверьте подключение кабелей, исправность монитора и видеокарты.
Схожесть: 0.82965595


In [11]:
# Задаём название модели RuBERT-tiny
MODEL_NAME = "cointegrated/rubert-tiny2"

# Загружаем токенайзер и модель RuBERT-tiny для получения эмбеддингов
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



# Функция получения эмбеддингов для текста
def get_embedding(text):
    # Токенизируем текст
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Берём среднее значение по токенам (mean pooling)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()[0]

# Загружаем датасет FAQ
dataset = load_dataset("json", data_files="computer_faq_extended_dataset.json")["train"]

# Строим базу: для каждого вопроса сохраняем эмбеддинг и ответ
faq_database = []
for item in dataset:
    question_text = item["вопрос"]
    answer_text = item["ответ"]
    emb = get_embedding(question_text)
    faq_database.append({
        "question": question_text,
        "answer": answer_text,
        "embedding": emb
    })

# Загружаем метрики
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

# Функция вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"],
    }

# Функция для поиска ближайшего вопроса в базе с помощью косинусного сходства
def find_best_answer(user_question):
    user_emb = get_embedding(user_question)
    # Сравниваем эмбеддинг пользователя с эмбеддингами FAQ
    embeddings = np.array([entry["embedding"] for entry in faq_database])
    scores = cosine_similarity([user_emb], embeddings)[0]
    best_idx = int(np.argmax(scores))
    best_match = faq_database[best_idx]
    return best_match["answer"], scores[best_idx]

# Функция для обработки запроса
def answer_question(user_question):
    answer, score = find_best_answer(user_question)
    return answer, score

# Пример использования
if __name__ == "__main__":
    user_question = "Почему компьютер не включается?"
    answer, score = answer_question(user_question)
    print("Вопрос:", user_question)
    print("Найденный ответ:", answer)
    print("Схожесть:", score)

Вопрос: Почему компьютер не включается?
Найденный ответ: Проверьте кабели, блок питания и исправность розетки.
Схожесть: 1.0


In [33]:
# Задаём название модели RuBERT-tiny
MODEL_NAME = "cointegrated/rubert-tiny2"

# Загружаем токенайзер и модель RuBERT-tiny для получения эмбеддингов
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Загружаем метрики
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

# Функция получения эмбеддингов для текста
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()[0]

# Загружаем датасет FAQ
dataset = load_dataset("json", data_files="computer_faq_extended_dataset.json")["train"]

# Строим базу: для каждого вопроса сохраняем эмбеддинг и ответ
faq_database = []
for item in dataset:
    question_text = item["вопрос"]
    answer_text = item["ответ"]
    emb = get_embedding(question_text)
    faq_database.append({
        "question": question_text,
        "answer": answer_text,
        "embedding": emb
    })

# Функция для поиска ближайшего вопроса в базе с помощью косинусного сходства
def find_best_answer(user_question, top_n=3, threshold=0.5):
    user_emb = get_embedding(user_question)
    embeddings = np.array([entry["embedding"] for entry in faq_database])
    scores = cosine_similarity([user_emb], embeddings)[0]

    # Берём top_n наиболее похожих вопросов
    top_indices = np.argsort(scores)[-top_n:][::-1]
    top_matches = [(faq_database[i]["answer"], scores[i]) for i in top_indices if scores[i] > threshold]

    if not top_matches:
        return "Извините, я не смог найти подходящий ответ.", 0.0

    # Выбираем ответ с наибольшим средним баллом или наиболее частый
    best_answer = max(top_matches, key=lambda x: x[1])[0]
    return best_answer, max(top_matches, key=lambda x: x[1])[1]

# Функция для обработки запроса
def answer_question(user_question):
    answer, score = find_best_answer(user_question)
    return answer, score

# Функция вычисления метрик
def compute_metrics():
    similarities = []
    for item in dataset:
        predicted_answer, _ = find_best_answer(item["вопрос"])
        real_emb = get_embedding(item["ответ"])
        pred_emb = get_embedding(predicted_answer)
        similarity = cosine_similarity([real_emb], [pred_emb])[0][0]
        similarities.append(similarity)
    return {
        "f1": f1_metric.compute(predictions=similarities, references=[1.0] * len(similarities), average="weighted")["f1"],
        "precision": precision_metric.compute(predictions=similarities, references=[1.0] * len(similarities), average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=similarities, references=[1.0] * len(similarities), average="weighted")["recall"],
    }

# Пример использования
if __name__ == "__main__":
    user_question = "Почему компьютер не включается?"
    answer, score = answer_question(user_question)
    print("Вопрос:", user_question)
    print("Найденный ответ:", answer)
    print("Схожесть:", score)
    
    # Вычисление метрик
    metrics = compute_metrics()
    print("Метрики:", metrics)

Вопрос: Почему компьютер не включается?
Найденный ответ: Проверьте кабели, блок питания и исправность розетки.
Схожесть: 1.0
Метрики: {'f1': 0.8177777777777778, 'precision': 1.0, 'recall': 0.6917293233082706}


C:\Users\alexr\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
if __name__ == "__main__":
    user_question = "Я удалил файлы, как восстановить?"
    answer, score = answer_question(user_question)
    print("Вопрос:", user_question)
    print("Найденный ответ:", answer)
    print("Схожесть:", score)

Вопрос: Я удалил файлы, как восстановить?
Найденный ответ: Используйте программы Recuva или встроенные средства восстановления.
Схожесть: 0.78735596


In [1]:
import json
import chromadb
from chromadb.utils import embedding_functions
from llama_cpp import Llama

# 1. Загрузка данных
with open('computer_faq_extended_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Инициализация ChromaDB
client = chromadb.PersistentClient(path="chroma_db")
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="paraphrase-multilingual-MiniLM-L12-v2"
)
collection = client.get_or_create_collection(
    name="computer_faq",
    embedding_function=embedding_func,
)

# 3. Добавление данных в векторную базу
collection.add(
    documents=[item["ответ"] for item in data],
    ids=[str(i) for i in range(len(data))],
    metadatas=[{"question": item["вопрос"]} for item in data]
)

# 4. Сохранение модели (в данном случае - только векторная база)
print("Векторная база знаний успешно создана!")

C:\Users\alexr\anaconda3\envs\my_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Векторная база знаний успешно создана!


In [5]:
import json
import chromadb
from chromadb.utils import embedding_functions
from llama_cpp import Llama

# Инициализация
client = chromadb.PersistentClient(path="chroma_db")
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction("paraphrase-multilingual-MiniLM-L12-v2")
collection = client.get_or_create_collection("computer_faq")

# Загрузка Llama3
llm = Llama(
    model_path="models/Llama3-Instruct-8B-RSPO.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=4
)

def get_answer(question):
    # Шаг 1: Поиск в датасете
    results = collection.query(
        query_texts=[question],
        n_results=1
    )
    
    exact_answer = results["documents"][0][0] if results["documents"] else None
    
    # Шаг 2: Проверка схожести вопроса
    if results["distances"][0][0] < 0.3:  # Порог схожести
        return exact_answer
    
    # Шаг 3: Генерация ответа для новых вопросов
    context = "\n".join(results["documents"][0])
    prompt = f"""Ты IT-специалист. Ответь на вопрос, используя контекст:
    
    Вопрос: {question}
    Контекст: {context}
    
    Ответ (1-2 предложения):"""
    
    output = llm(prompt, max_tokens=150, temperature=0.2)
    return output["choices"][0]["text"].strip()

# Примеры
print(get_answer("Почему компьютер не включается?"))  # Ответ из датасета


llama_model_loader: loaded meta data with 38 key-value pairs and 291 tensors from models/Llama3-Instruct-8B-RSPO.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3 Instruct 8B RSPO
llama_model_loader: - kv   3:                       general.organization str              = Li11111
llama_model_loader: - kv   4:                           general.finetune str              = Instruct-RSPO
llama_model_loader: - kv   5:                           general.basename str              = Llama-3
llama_model_loader: - kv   6:                         general.size_label str              = 8B
llama_model_loader: - kv   7:       

Необходимо проверить физическое состояние питания компьютера и его блоки питания. Если проблема не связана с ними, можно предположить проблему с файловой системой или заражением вирусом. В этом случае попытка использования программы Recuva для восстановления данных или команда `attrib` для проверки и изменений атрибутов файлов может помочь диагностировать и решить проблему с включением компьютера.



llama_print_timings:        load time =    5610.05 ms
llama_print_timings:      sample time =      38.34 ms /    99 runs   (    0.39 ms per token,  2582.29 tokens per second)
llama_print_timings: prompt eval time =    3679.21 ms /    59 tokens (   62.36 ms per token,    16.04 tokens per second)
llama_print_timings:        eval time =   15254.05 ms /    98 runs   (  155.65 ms per token,     6.42 tokens per second)
llama_print_timings:       total time =   19743.96 ms /   157 tokens


Необходимо проверить файловые атрибуты на диске X:, возможно файлы или папки имеют скрытый ('h'), системный ('s') или защищенный ('r') статус, что мешает их отображению на мониторе. Используемая команда 'attrib' или программа Recuva позволяют изменить эти атрибуты и сделать файлы доступными для просмотра на экране.


In [6]:
print(get_answer("Почему нет изображения на мониторе?"))  # Генерация нового ответа

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5610.05 ms
llama_print_timings:      sample time =      45.02 ms /   116 runs   (    0.39 ms per token,  2576.92 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   18229.32 ms /   116 runs   (  157.15 ms per token,     6.36 tokens per second)
llama_print_timings:       total time =   19136.80 ms /   117 tokens


Необходимо проверить файловые атрибуты на диске X:, возможно файлы или папки имеют скрытый ('h'), системный ('s') или защищенный ('r') статус, что мешает их отображению на мониторе. Используемая команда 'attrib' или программа Recuva помогут обнаружить и изменить эти атрибуты, если необходимо. Затем нужно повторить попытку просмотра содержимого диска для отображения файлов и изображений на мониторе.


In [1]:
import json
import chromadb
from chromadb.utils import embedding_functions
from llama_cpp import Llama

# Инициализация
client = chromadb.PersistentClient(path="chroma_db")
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction("paraphrase-multilingual-MiniLM-L12-v2")
collection = client.get_or_create_collection("computer_faq")

# Загрузка Llama3
llm = Llama(
    model_path="models/Llama3-Instruct-8B-RSPO.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=4
)

def get_answer(question):
    # Шаг 1: Поиск релевантных ответов
    results = collection.query(
        query_texts=[question],
        n_results=3  # Берем больше результатов для лучшего контекста
    )
    
    # Шаг 2: Проверка релевантности (более строгий порог)
    if results["distances"][0][0] < 0.5:  # Более консервативный порог
        best_answer = results["documents"][0][0]
        best_question = results["metadatas"][0][0]["question"]
        
        # Дополнительная проверка схожести вопроса
        if question.lower() in best_question.lower() or best_question.lower() in question.lower():
            return best_answer
    
    # Шаг 3: Подготовка контекста для генерации
    context = "\n".join([
        f"Вопрос: {q['question']}\nОтвет: {a}" 
        for q, a in zip(results["metadatas"][0], results["documents"][0])
    ])
    
    # Шаг 4: Генерация ответа с четкими инструкциями
    prompt = f"""Ты опытный IT-специалист. Ответь на вопрос пользователя кратко и по делу.
Если вопрос не связан с IT, вежливо сообщи об этом.

Вопрос: {question}

Релевантные знания из базы:
{context}

Ответ (1-2 предложения, только по существу):"""
    
    output = llm(
        prompt,
        max_tokens=150,
        temperature=0.1,  # Меньше "креатива"
        stop=["\n", "Вопрос:", "Контекст:"]
    )
    
    return output["choices"][0]["text"].strip()

# Примеры
print(get_answer("Почему нет изображения на мониторе?"))
print(get_answer("Как починить принтер?"))
print(get_answer("Что делать, если компьютер не включается?"))

C:\Users\alexr\anaconda3\envs\my_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 38 key-value pairs and 291 tensors from models/Llama3-Instruct-8B-RSPO.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3 Instruct 8B RSPO
llama_model_loader: - kv   3:                       general.organization str              = Li11111
llama_model_loader: - kv   4:                           general.finetune str 

Монитор может не отображать изображение из-за ошибки драйвера видеокарты, проблемы со соединением видеоадаптера или физического повреждения монитора или кабеля VGA/HDMI. Проверьте соответствующие компоненты и обновите драйвер видеокарты, если необходимо. Если проблема persists, обратитесь к технической поддержке или проверьте монитор на повреждения.



llama_print_timings:        load time =   12542.39 ms
llama_print_timings:      sample time =      30.04 ms /    75 runs   (    0.40 ms per token,  2497.00 tokens per second)
llama_print_timings: prompt eval time =    9226.39 ms /   167 tokens (   55.25 ms per token,    18.10 tokens per second)
llama_print_timings:        eval time =   11909.29 ms /    74 runs   (  160.94 ms per token,     6.21 tokens per second)
llama_print_timings:       total time =   21742.13 ms /   241 tokens
Llama.generate: prefix-match hit


Для починки принтера проверьте соединение кабелей, убедитесь в наличии подходящего драйвера и обновите его если необходимости требует. Если проблема persists, попробуйте очистку принтерной головки или обратитесь к инструкции по техническому обслуживанию или контактуруте производителя поддержки.



llama_print_timings:        load time =   12542.39 ms
llama_print_timings:      sample time =      40.04 ms /   101 runs   (    0.40 ms per token,  2522.35 tokens per second)
llama_print_timings: prompt eval time =    9952.34 ms /   181 tokens (   54.99 ms per token,    18.19 tokens per second)
llama_print_timings:        eval time =   16198.01 ms /   100 runs   (  161.98 ms per token,     6.17 tokens per second)
llama_print_timings:       total time =   27009.71 ms /   281 tokens


Если компьютер не включается, проверьте источник питания (заряд батареи или подключение к сети), проверьте блок питания и кабели подключения к монитору и периферийным устройствам. В случае необходимости выполните сброс устройства или обратитесь к инструкции по восстановлению из заводских настроек производителя. Если проблема persists, рекомендуется обратиться к специалисту или контактировать с технической поддержкой производителя.


In [6]:
print(get_answer("Микрофон выключился в неожиданный момент"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   12542.39 ms
llama_print_timings:      sample time =      25.64 ms /    65 runs   (    0.39 ms per token,  2535.00 tokens per second)
llama_print_timings: prompt eval time =   10022.85 ms /   184 tokens (   54.47 ms per token,    18.36 tokens per second)
llama_print_timings:        eval time =   10070.51 ms /    64 runs   (  157.35 ms per token,     6.36 tokens per second)
llama_print_timings:       total time =   20617.44 ms /   248 tokens


Файловый менеджер или программка для восстановления файлов (например, Recuva) помогут найти и вернуть пропавшие файлы с флешки. Проверьте подключение флешки и состояние ее файловой системы перед попыткой восстановления данных.


In [44]:
import json
import chromadb
from chromadb.utils import embedding_functions
from llama_cpp import Llama

# Инициализация
client = chromadb.PersistentClient(path="chroma_db")
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction("paraphrase-multilingual-MiniLM-L12-v2")
collection = client.get_or_create_collection("computer_faq")

# Загрузка Llama3 с оптимизированными параметрами
llm = Llama(
    model_path="models/Llama3-Instruct-8B-RSPO.Q4_K_M.gguf",
    n_ctx=4096,  # Увеличенный контекст
    n_threads=6,
    n_gpu_layers=50  # Если есть GPU
)

def get_answer(question):
    # 1. Поиск релевантного контекста (даже слабые совпадения)
    results = collection.query(
        query_texts=[question],
        n_results=5,
        include=["documents", "metadatas"]
    )
    
    # 2. Формирование динамического контекста
    context = "Возможные решения из базы знаний:\n"
    for meta, doc in zip(results["metadatas"][0], results["documents"][0]):
        context += f"- {meta['question']}: {doc}\n"
    
    # 3. Универсальный промпт с "страховкой" для любых вопросов
    prompt = f"""Ты опытный IT-специалист. Отвечай на вопросы, используя:
1. Контекст из базы знаний (если релевантен)
2. Собственные экспертные знания
3. Логические выводы

Контекст:
{context}

Вопрос: {question}

Структура ответа:
1. Основное решение (1-2 предложения)
2. Дополнительные проверки (если нужно)
3. Альтернативные варианты (если применимо)

Ответ:"""
    
    # Первичная генерация
    output = llm(prompt, max_tokens=500, temperature=0.3)
    answer = output["choices"][0]["text"].strip()

    # Принудительное завершение если ответ неполный
    if not answer.count('\n') >= 3:  # Менее 3 пунктов
        continuation = llm(
            f"Заверши ответ:\n{answer}",
            max_tokens=200,
            temperature=0.1
        )
        answer += "\n" + continuation["choices"][0]["text"].strip()

    return answer

llama_model_loader: loaded meta data with 38 key-value pairs and 291 tensors from models/Llama3-Instruct-8B-RSPO.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3 Instruct 8B RSPO
llama_model_loader: - kv   3:                       general.organization str              = Li11111
llama_model_loader: - kv   4:                           general.finetune str              = Instruct-RSPO
llama_model_loader: - kv   5:                           general.basename str              = Llama-3
llama_model_loader: - kv   6:                         general.size_label str              = 8B
llama_model_loader: - kv   7:       

In [22]:
print(get_answer("Микрофон выключился в неожиданный момент"))


llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      46.91 ms /   121 runs   (    0.39 ms per token,  2579.68 tokens per second)
llama_print_timings: prompt eval time =   18819.18 ms /   345 tokens (   54.55 ms per token,    18.33 tokens per second)
llama_print_timings:        eval time =   19245.16 ms /   120 runs   (  160.38 ms per token,     6.24 tokens per second)
llama_print_timings:       total time =   39050.44 ms /   465 tokens


При неожиданном выключении микрофона в Windows рекомендуется проверить его свойства в Панели управления звуком (например, через «Сonus» или «Звук» в Панели управления). Перейдите в раздел «Recording devices» или «Микрофоны», найдите выключенный микрофон и убедитесь, что он не отключен или не заблокирован. Если проблема persists, проверьте драйвера звукового адаптера и обновите их, если необходимости требует.


In [37]:
print(get_answer("Как настроить RGB-подсветку на нестандартной клавиатуре?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18428.48 ms
llama_print_timings:      sample time =      82.70 ms /   212 runs   (    0.39 ms per token,  2563.39 tokens per second)
llama_print_timings: prompt eval time =   14720.60 ms /   269 tokens (   54.72 ms per token,    18.27 tokens per second)
llama_print_timings:        eval time =   33750.27 ms /   211 runs   (  159.95 ms per token,     6.25 tokens per second)
llama_print_timings:       total time =   50250.83 ms /   480 tokens


Для настройки RGB-подсветки на нестандартной клавиатуре рекомендуется использовать специализированное программное обеспечение от производителя клавиатуры или третьих разработчиков,such as Keyboard Lighting Software или similar tools.

Перед настройкой проверьте,是否 клавиатура поддерживает RGB-подсветку и имеет соответствующие программные инструменты для ее управления. В противном случае может потребоваться поиск альтернативного решения или проверка документации производителя клавиатуры для настройки подсветки.

В качестве альтернативы можно попробовать использовать всеобщие инструменты для управления освещением, такие как RGB-control software или даже некоторые медиаплэеры с поддержкой освещения для клавиатур (если они совместимы с вашей клавиатурой). Однако эффективность таких решений может зависеть от конкретной модели клавиатуры и ее программного обеспечения.


In [25]:
print(get_answer("Почему моя беспроводная мышь дергается при работе на стеклянном столе?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      20.18 ms /    52 runs   (    0.39 ms per token,  2576.30 tokens per second)
llama_print_timings: prompt eval time =    9573.38 ms /   172 tokens (   55.66 ms per token,    17.97 tokens per second)
llama_print_timings:        eval time =    8042.15 ms /    51 runs   (  157.69 ms per token,     6.34 tokens per second)
llama_print_timings:       total time =   18028.05 ms /   223 tokens


При работе беспроводной мыши на стеклянном столе может возникать дергание из-за слабого сигнала радиосигнала к приемнику мыши или помехи от отражения радиоволн от поверхности стола.


In [26]:
print(get_answer("Как перенести Windows с HDD на NVMe SSD без переустановки?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      26.26 ms /    69 runs   (    0.38 ms per token,  2627.37 tokens per second)
llama_print_timings: prompt eval time =   14104.98 ms /   256 tokens (   55.10 ms per token,    18.15 tokens per second)
llama_print_timings:        eval time =   10734.17 ms /    68 runs   (  157.86 ms per token,     6.33 tokens per second)
llama_print_timings:       total time =   25399.79 ms /   324 tokens


Для переноса Windows с HDD на NVMe SSD без переустановки рекомендуется использовать инструмент «Disk Clone» или «Disk Imaging» программы Acronis True Image или EaseUS Todo Backup. Эти программы позволяют создать точную копию жесткого диска HDD и записать ее на новый SSD NVMe.


In [27]:
print(get_answer("Почему в Zoom мое лицо выглядит размытым при виртуальном фоне?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      19.50 ms /    51 runs   (    0.38 ms per token,  2615.92 tokens per second)
llama_print_timings: prompt eval time =   12404.23 ms /   225 tokens (   55.13 ms per token,    18.14 tokens per second)
llama_print_timings:        eval time =    7906.86 ms /    50 runs   (  158.14 ms per token,     6.32 tokens per second)
llama_print_timings:       total time =   20717.98 ms /   275 tokens


При появлении размытого изображения лица в Zoom при использовании виртуального фона возможной причиной является низкая разрешающая способность видеокамеры или неоптимальная настройка камеры в системе.


In [28]:
print(get_answer("Что делать, если ноутбук заряжается только в одном определенном положении кабеля?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      27.36 ms /    70 runs   (    0.39 ms per token,  2558.85 tokens per second)
llama_print_timings: prompt eval time =   13191.23 ms /   238 tokens (   55.43 ms per token,    18.04 tokens per second)
llama_print_timings:        eval time =   10915.43 ms /    69 runs   (  158.19 ms per token,     6.32 tokens per second)
llama_print_timings:       total time =   24676.71 ms /   307 tokens


При заряжении ноутбука только в одном определенном положении кабеля может возникнуть проблема с контактами зажимов или повреждением гибкого кабеля. Основное решение consists in проверке и очистке контактов зажимов от пыли и грязи или заменой поврежденного кабеля.


In [29]:
print(get_answer("Как настроить цветопередачу монитора для профессионального фоторедактирования?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      17.65 ms /    46 runs   (    0.38 ms per token,  2605.94 tokens per second)
llama_print_timings: prompt eval time =    4730.77 ms /    80 tokens (   59.13 ms per token,    16.91 tokens per second)
llama_print_timings:        eval time =    7268.62 ms /    45 runs   (  161.52 ms per token,     6.19 tokens per second)
llama_print_timings:       total time =   12380.63 ms /   125 tokens


Для настройки цветопередачи монитора для профессионального фоторедактирования рекомендуется изменить параметры цветового профиля в настройках монитора или видеокарты.


In [38]:
print(get_answer("Почему Bluetooth-наушники подключаются к телефону, но не видят Windows?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18428.48 ms
llama_print_timings:      sample time =     106.87 ms /   272 runs   (    0.39 ms per token,  2545.15 tokens per second)
llama_print_timings: prompt eval time =   15100.96 ms /   280 tokens (   53.93 ms per token,    18.54 tokens per second)
llama_print_timings:        eval time =   43126.36 ms /   271 runs   (  159.14 ms per token,     6.28 tokens per second)
llama_print_timings:       total time =   60500.39 ms /   551 tokens


При подключении Bluetooth-наушников к телефону они устанавливают парковку с устройством и обмениваются информацией о подключении. Однако в Windows может возникнуть проблема с обнаружением устройств Bluetooth из-за следующих причин:

Основное решение: Проверьте панель устройств Bluetooth в Windows («Устройства» → «Бluetooth» или через центр управления Bluetooth в Панели управления). Убедитесь, что Bluetooth-сервис активен и устройство наушников отображается в списке доступных устройств.

Дополнительные проверки: Обратите внимание на уровень сигнала Bluetooth-сигнала в наушниках и на компьютере. Если сигнал слабый, попытайтесь переместить устройство ближе к компьютеру или изменить позицию антенны в наушниках.

Альтернативные варианты: Если проблема persists, можно попытаться обновить драйвера Bluetooth в системе или деинсталлировать и再инсталлировать устройство наушников в Windows Device Manager. В крайнем случае, обратитесь к производителю наушников за поддержкой или обновлениями драйвер

In [36]:
print(get_answer("Как раздать интернет с ПК через Ethernet-кабель на Smart TV?"))


llama_print_timings:        load time =   18428.48 ms
llama_print_timings:      sample time =     106.79 ms /   276 runs   (    0.39 ms per token,  2584.49 tokens per second)
llama_print_timings: prompt eval time =   18428.08 ms /   339 tokens (   54.36 ms per token,    18.40 tokens per second)
llama_print_timings:        eval time =   43793.24 ms /   275 runs   (  159.25 ms per token,     6.28 tokens per second)
llama_print_timings:       total time =   64512.53 ms /   614 tokens


Для раздачи интернета с ПК через Ethernet-кабель на Smart TV необходимо:

Основное решение: подключить Ethernet-кабель от компьютера к Smart TV и настроить сетевой шлюз или DHCP-сервер на ПК для обеспечения доступа к интернету на телевизоре.

Дополнительные проверки: убедитесь, что Ethernet-кабель правильно подключен к обоим устройствам (ПК и Smart TV), а также что сетевая карта на ПК и Smart TV поддерживают соответствующие протоколы и стандарты (например, DHCP и IPv4 или IPv6).

Альтернативные варианты: если у вас доступен WiFi на ПК и Smart TV, можно использовать беспроводной способ подключения к интернету и SKIP Ethernet-кабель. В этом случае настройте WiFi-сеть на ПК и Smart TV соответственно. Если у вас нет доступа к настройкам Smart TV или возникнут проблемы с Ethernet-соединением, можно использовать дополнительное сетевое устройство,such as a wireless router or a network switch, для обеспечения доступа к интернету от ПК к Smart TV. Однако это может потребовать дополнительных нас

In [32]:
print(get_answer("Почему после обновления BIOS сбросились настройки разгона?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      25.33 ms /    65 runs   (    0.39 ms per token,  2566.53 tokens per second)
llama_print_timings: prompt eval time =    7834.99 ms /   141 tokens (   55.57 ms per token,    18.00 tokens per second)
llama_print_timings:        eval time =   10178.93 ms /    64 runs   (  159.05 ms per token,     6.29 tokens per second)
llama_print_timings:       total time =   18540.40 ms /   205 tokens


При сбросе настроек разгона после обновления BIOS возможной причиной является изменение конфигурации утилитами фирмы-производителя или автоматическим сбросом заводских значений при обновлении биоса для обеспечения стабильной работы системы.


In [33]:
print(get_answer("Как сделать так, чтобы второй монитор не отключался при сворачивании игр?"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18819.44 ms
llama_print_timings:      sample time =      18.98 ms /    48 runs   (    0.40 ms per token,  2529.11 tokens per second)
llama_print_timings: prompt eval time =   14657.76 ms /   268 tokens (   54.69 ms per token,    18.28 tokens per second)
llama_print_timings:        eval time =    7429.70 ms /    47 runs   (  158.08 ms per token,     6.33 tokens per second)
llama_print_timings:       total time =   22476.44 ms /   315 tokens


Для предотвращения отключения второго монитора при сворачивании игр рекомендуется использовать функцию «Постоянное отображение экрана» в настройках видеокарты или монитора.


In [45]:
print(get_answer("Почему Thunderbolt-док-станция не заряжает ноутбук, но устройства работают?"))


llama_print_timings:        load time =   19182.00 ms
llama_print_timings:      sample time =      89.75 ms /   229 runs   (    0.39 ms per token,  2551.45 tokens per second)
llama_print_timings: prompt eval time =   19181.71 ms /   355 tokens (   54.03 ms per token,    18.51 tokens per second)
llama_print_timings:        eval time =   36316.63 ms /   228 runs   (  159.28 ms per token,     6.28 tokens per second)
llama_print_timings:       total time =   57532.46 ms /   583 tokens


При отсутствии зарядки ноутбука от Thunderbolt-док-станции, но при нормальной работе других устройств, вероятно, проблема связана с неисправностью зарядного модуля или несовместимостью станции с конкретным типом ноутбука или его зарядной системы.

Дополнительная проверка: Проверьте соответствие станции и ноутбука по следующим параметрам:
* Тип зарядки (USB-C Power Delivery или Thunderbolt Power Delivery);
* Максимальная мощность заряда;
* Поддерживаемые протоколы зарядки.

Альтернативный вариант: Возможно, проблема вызвана настройкой станции или конфликтом с другими устройствами, подключенными к ней. Попробуйте отключить другие устройства и проверить зарядку ноутбука отдельно. Если проблема persists, рекомендуется обратиться к производителю станции или контактировать с технической поддержкой для диагностического обследования и решения проблемы.
